In [15]:
# -*- coding: utf-8 -*-
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.optimizers import SGD
import math

In [16]:
# 이미지 데이터 경로 설정
train_path = r'/content/drive/MyDrive/음식'
valid_path = r'/content/drive/MyDrive/음식'

test_path = r'/content/drive/MyDrive/음식'

In [17]:
# 이미지 크기 설정
img_width, img_height = 224, 224

# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.5,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.5,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# 배치 크기 조정
batch_size = 64

train_generator = train_datagen.flow_from_directory(train_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')



Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# VGG16 모델 로딩 및 학습 레이어 제거
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
last_layer = vgg16_model.output
last_layer = Flatten()(last_layer)
last_layer = Dense(256, activation='relu')(last_layer)
last_layer = Dropout(0.5)(last_layer)
output_layer = Dense(train_generator.num_classes, activation='softmax')(last_layer)

model = Model(inputs=vgg16_model.input, outputs=output_layer)


In [20]:
# 미세 조정을 위한 학습 설정
for layer in model.layers[:15]:
    layer.trainable = False

# 학습률 조정
lr = 0.01
sgd = SGD(lr=lr, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [21]:
# 모델 학습 및 평가
steps_per_epoch = math.floor(train_generator.samples / train_generator.batch_size)
validation_steps = math.floor(valid_generator.samples / valid_generator.batch_size)
test_steps = math.ceil(test_generator.samples / test_generator.batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)

scores = model.evaluate_generator(test_generator, steps=test_steps)
print("Test accuracy: %.2f%%" % (scores[1]*100))


<ipython-input-21-19dca02107cf>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)


Epoch 1/5
6/6 [==============================] - 618s 111s/step - loss: 2.6273 - accuracy: 0.2173 - val_loss: 1.3808 - val_accuracy: 0.3125
Epoch 2/5
6/6 [==============================] - 609s 118s/step - loss: 1.3883 - accuracy: 0.3244 - val_loss: 1.3829 - val_accuracy: 0.2682
Epoch 3/5
6/6 [==============================] - 566s 102s/step - loss: 1.8653 - accuracy: 0.2768 - val_loss: 1.3863 - val_accuracy: 0.2526
Epoch 4/5
6/6 [==============================] - 558s 108s/step - loss: 1.3861 - accuracy: 0.2470 - val_loss: 1.3867 - val_accuracy: 0.2500
Epoch 5/5
6/6 [==============================] - 613s 111s/step - loss: 1.3859 - accuracy: 0.2560 - val_loss: 1.3865 - val_accuracy: 0.2526


<ipython-input-21-19dca02107cf>:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, steps=test_steps)


Test accuracy: 25.00%
